In [1]:
import os, requests, uuid, json

In [2]:
subscription_key = '51614b38d9774265a077a2f69d3ea320'

# Pure Translation

In [14]:
base_url = 'https://api.cognitive.microsofttranslator.com'
path = '/translate?api-version=3.0'
params = '&to=de&to=it'
constructed_url = base_url + path + params
headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}
body = [{
    'text' : 'Hello World!'
}]
request = requests.post(constructed_url, headers=headers, json=body)
response = request.json()
print(json.dumps(response, sort_keys=True, indent=4, ensure_ascii=False, separators=(',', ': ')))

[
    {
        "detectedLanguage": {
            "language": "en",
            "score": 1.0
        },
        "translations": [
            {
                "text": "Hallo Welt!",
                "to": "de"
            },
            {
                "text": "Salve, mondo!",
                "to": "it"
            }
        ]
    }
]


# Alternative translations

In [22]:
base_url = 'https://api.cognitive.microsofttranslator.com'
path = '/dictionary/lookup?api-version=3.0'
params = '&from=en&to=ru';
headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}
constructed_url = base_url + path + params

In [23]:
# same headers as before
body = [{
    'text': 'faithfully'
}]

In [24]:
request = requests.post(constructed_url, headers=headers, json=body)
response = request.json()

In [25]:
for item in response[0]['translations']:
    print('Translation: ' + item['displayTarget'])
    print('Confidence: ' + str(item['confidence']))
    print('POS: ' + item['posTag'])
    print('\n')

response

Translation: добросовестно
Confidence: 0.3946
POS: ADV


Translation: преданно
Confidence: 0.3274
POS: ADV


Translation: верно
Confidence: 0.1402
POS: ADV


Translation: искренне
Confidence: 0.1378
POS: ADV




[{'displaySource': 'faithfully',
  'normalizedSource': 'faithfully',
  'translations': [{'backTranslations': [{'displayText': 'faithfully',
      'frequencyCount': 61,
      'normalizedText': 'faithfully',
      'numExamples': 1},
     {'displayText': 'conscientiously',
      'frequencyCount': 38,
      'normalizedText': 'conscientiously',
      'numExamples': 5},
     {'displayText': 'diligently',
      'frequencyCount': 10,
      'normalizedText': 'diligently',
      'numExamples': 0}],
    'confidence': 0.3946,
    'displayTarget': 'добросовестно',
    'normalizedTarget': 'добросовестно',
    'posTag': 'ADV',
    'prefixWord': ''},
   {'backTranslations': [{'displayText': 'faithfully',
      'frequencyCount': 30,
      'normalizedText': 'faithfully',
      'numExamples': 5},
     {'displayText': 'devotedly',
      'frequencyCount': 9,
      'normalizedText': 'devotedly',
      'numExamples': 2}],
    'confidence': 0.3274,
    'displayTarget': 'преданно',
    'normalizedTarget': 'пре

# Get Languages

In [4]:
headers = {
    'Ocp-Apim-Subscription-Key': subscription_key,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}
base_url = 'https://api.cognitive.microsofttranslator.com'
path = '/languages?api-version=3.0'
constructed_url = base_url + path
request = requests.get(constructed_url, headers=headers)
response = request.json()

{'dictionary': {'af': {'dir': 'ltr',
   'name': 'Afrikaans',
   'nativeName': 'Afrikaans',
   'translations': [{'code': 'en',
     'dir': 'ltr',
     'name': 'English',
     'nativeName': 'English'}]},
  'ar': {'dir': 'rtl',
   'name': 'Arabic',
   'nativeName': 'العربية',
   'translations': [{'code': 'en',
     'dir': 'ltr',
     'name': 'English',
     'nativeName': 'English'}]},
  'bg': {'dir': 'ltr',
   'name': 'Bulgarian',
   'nativeName': 'Български',
   'translations': [{'code': 'en',
     'dir': 'ltr',
     'name': 'English',
     'nativeName': 'English'}]},
  'bn': {'dir': 'ltr',
   'name': 'Bangla',
   'nativeName': 'বাংলা',
   'translations': [{'code': 'en',
     'dir': 'ltr',
     'name': 'English',
     'nativeName': 'English'}]},
  'bs': {'dir': 'ltr',
   'name': 'Bosnian (Latin)',
   'nativeName': 'bosanski (latinica)',
   'translations': [{'code': 'en',
     'dir': 'ltr',
     'name': 'English',
     'nativeName': 'English'}]},
  'ca': {'dir': 'ltr',
   'name': 'Catalan

# Parse CSV

In [3]:
from pathlib import Path
from lxml import etree
import pandas as pd

In [14]:
# Translate to L1 using Microsoft Translation API.
# Source: `data/no_spell_errors.csv`, Target: `data/ms_translated_all.csv`

source = '../data/no_spell_errors.csv'
pd_dataset = pd.read_csv(source)
pd_dataset.head()

,language,type,i,c,id
0,Russian,RV,propose,suggest,person_624
1,Russian,RV,suppose,think,person_624
2,Russian,RN,people,public,person_624
3,Russian,RY,faithfully,sincerely,person_624
4,Russian,RJ,common,familiar,person_624


In [5]:
dataset = {}

# languages: ['Russian', 'Turkish', 'German', 'Japanese', 'Spanish', 'Polish', 'Italian',
# 'Catalan', 'Korean', 'French', 'Chinese', 'Portuguese', 'Swedish', 'Greek'
# , 'Thai', 'Dutch']
def create_store(dataset, language):
    if (language in dataset):
        return
    
    dataset[language] = {}
    
    # mapping from https://docs.microsoft.com/en-us/azure/cognitive-services/translator/language-support
    if (language == "Russian"):
        dataset[language]['code'] = 'ru'
        dataset[language]['type'] = 'neural'
    elif (language == "Turkish"):
        dataset[language]['code'] = 'tr'
        dataset[language]['type'] = 'neural'
    elif (language == "German"):
        dataset[language]['code'] = 'de'
        dataset[language]['type'] = 'neural'
    elif (language == "Japanese"):
        dataset[language]['code'] = 'ja'
        dataset[language]['type'] = 'neural'
    elif (language == "Spanish"):
        dataset[language]['code'] = 'es'
        dataset[language]['type'] = 'neural'
    elif (language == "Polish"):
        dataset[language]['code'] = 'pl'
        dataset[language]['type'] = 'neural'
    elif (language == "Italian"):
        dataset[language]['code'] = 'it'
        dataset[language]['type'] = 'neural'
    elif (language == "Catalan"):
        dataset[language]['code'] = 'ca'
        dataset[language]['type'] = 'statistical'
    elif (language == "Korean"):
        dataset[language]['code'] = 'ko'
        dataset[language]['type'] = 'neural'
    elif (language == "French"):
        dataset[language]['code'] = 'fr'
        dataset[language]['type'] = 'neural'
    elif (language == "Chinese"):
        # only hanz available (chinese simplified)
        dataset[language]['code'] = 'zh-Hans'
        dataset[language]['type'] = 'neural'
    elif (language == "Portuguese"):
        dataset[language]['code'] = 'pt'
        dataset[language]['type'] = 'neural'
    elif (language == "Swedish"):
        dataset[language]['code'] = 'sv'
        dataset[language]['type'] = 'neural'
    elif (language == "Greek"):
        dataset[language]['code'] = 'el'
        dataset[language]['type'] = 'neural'
    elif (language == "Thai"):
        dataset[language]['code'] = 'th'
        dataset[language]['type'] = 'neural'
    elif (language == "Dutch"):
        dataset[language]['code'] = 'nl'
        dataset[language]['type'] = 'neural'
    
    dataset[language]['data'] = []

def create_entry(row):
    return {'type': row['type'], 'i': row['i'], 'c': row['c'], 'id': row['id'], 'l1_i': '', 'l1_c': ''}

for index, item in pd_dataset.iterrows():
    if (item['language'] not in dataset):
        create_store(dataset, item['language'])
        
    dataset[item['language']]['data'].append(create_entry(item))

In [6]:
base_url = 'https://api.cognitive.microsofttranslator.com'
path = '/dictionary/lookup?api-version=3.0'
# Guide for POS: `N - Noun, J - Adjective, V - Verb, Y - Adverb`
anomaly_dict = []
def is_pos_match(api_tag, data_tag):
    if (data_tag == 'RN' and api_tag == 'NOUN'):
        return True
    if (data_tag == 'RJ' and api_tag == 'ADJ'):
        return True
    if (data_tag == 'RV' and api_tag == 'VERB'):
        return True
    if (data_tag == 'RY' and api_tag == 'ADV'):
        return True
    return False

def extract_translation(translation, pos_tag):
    pos_confidence, generic_confidence = -1, -1
    pos_alternative, generic_alternative = '', ''
    for alternative in translation['translations']:
        if (is_pos_match(alternative['posTag'], pos_tag) and alternative['confidence'] > pos_confidence):
            pos_alternative = alternative
            pos_confidence = alternative['confidence']
        elif (alternative['confidence'] > generic_confidence):
            generic_alternative = alternative
            generic_confidence = alternative['confidence']
    
    if (pos_alternative == '' and generic_alternative == ''):
        anomaly_dict.append({'translation': translation, 'pos': pos_tag})
        return ''
    elif (pos_alternative == ''):
        return generic_alternative['normalizedTarget']
    
    return pos_alternative['normalizedTarget']

def get_translations(data, start, end, language_code):
    params = '&from=en&to=' + language_code;
    constructed_url = base_url + path + params
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }
    body = []
    for item in data[start:end]:
        body.append({'Text': item['i']})
        body.append({'Text': item['c']})
    
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    
    index = 0
    for i in range(0, len(response), 2):
        data[start + index]['l1_i'] = extract_translation(response[i], data[start + index]['type'])
        data[start + index]['l1_c'] = extract_translation(response[i + 1], data[start + index]['type'])
        index += 1
        

In [7]:
# Create batches of data for a language, send to msft, and add response back to dict
def process_batch(dataset, language):
    data = dataset[language]['data']
    start = 0
    limit = len(data)
    batch_size = 5 # limit on msft api is 10
    end = start + batch_size
    
    incorrect_payload = []
    correct_payload = []
    while end < limit:
#         for item in data[start:end]:

#             incorrect_payload.append({'Text': item['i']})
#             correct_payload.append({'Text': item['c']})
        # Call API
        get_translations(dataset[language]['data'], start, end, dataset[language]['code'])
        incorrect_payload = []
        correct_payload = []
        start = end
        if (end + batch_size > limit):
            end = limit
        else:
            end += batch_size



In [8]:
from datetime import datetime
for key in dataset.keys():
    start = datetime.now()
    process_batch(dataset, key)
    print('Finished language: ' + key + ' in ' + str((datetime.now() - start)))

Finished language: Russian in 0:00:26.798887
Finished language: Turkish in 0:00:23.016081
Finished language: German in 0:00:19.911327
Finished language: Japanese in 0:00:18.758352
Finished language: Spanish in 0:00:56.343635
Finished language: Polish in 0:00:21.660337
Finished language: Italian in 0:00:24.467640
Finished language: Catalan in 0:00:24.834779
Finished language: Korean in 0:00:14.565863
Finished language: French in 0:01:00.311514
Finished language: Chinese in 0:00:22.857705
Finished language: Portuguese in 0:00:19.269247
Finished language: Swedish in 0:00:03.007272
Finished language: Greek in 0:00:25.610024
Finished language: Thai in 0:00:15.584518
Finished language: Dutch in 0:00:00.337173


In [17]:
pre_dataframe = []
for key in dataset.keys():
    for row in dataset[key]['data']:
        pre_dataframe.append({**row, **{'language': key}})

In [21]:
new_dataframe = pd.DataFrame(pre_dataframe)

In [23]:
new_dataframe.head()

,c,i,id,l1_c,l1_i,language,type
0,suggest,propose,person_624,предложить,предлагаем,Russian,RV
1,think,suppose,person_624,думаю,предположим,Russian,RV
2,public,people,person_624,общественности,людей,Russian,RN
3,sincerely,faithfully,person_624,искренне,добросовестно,Russian,RY
4,familiar,common,person_624,знакомы,общие,Russian,RJ


In [24]:
new_dataframe.to_csv('../data/ms_translated_all.csv', index = False)